<a href="https://colab.research.google.com/github/Livia711/livia/blob/main/Vexpenses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as bibliotecas necessárias
import sqlite3
import pandas as pd
from IPython.display import display, HTML

# Conectando ao banco de dados (em memória)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 1. Criação das tabelas
cursor.execute("""
CREATE TABLE clientes (
  id INTEGER PRIMARY KEY,
  nome TEXT,
  data_criacao DATE
);
""")

cursor.execute("""
CREATE TABLE pedidos (
  id INTEGER PRIMARY KEY,
  cliente_id INTEGER,
  data_pedido DATE,
  valor_total REAL,
  FOREIGN KEY (cliente_id) REFERENCES clientes(id)
);
""")

cursor.execute("""
CREATE TABLE itens_pedido (
  id INTEGER PRIMARY KEY,
  pedido_id INTEGER,
  produto TEXT,
  quantidade INTEGER,
  preco_unitario REAL,
  FOREIGN KEY (pedido_id) REFERENCES pedidos(id)
);
""")

# 2. Inserção de dados fictícios
cursor.execute("""
INSERT INTO clientes (id, nome, data_criacao) VALUES
(1, 'Lívia', '2024-01-10'),
(2, 'Eduardo', '2024-02-15'),
(3, 'Ana', '2024-03-05'),
(4, 'Carlos', '2024-03-15'),
(5, 'Mariana', '2024-04-01');
""")

cursor.execute("""
INSERT INTO pedidos (id, cliente_id, data_pedido, valor_total) VALUES
(1, 1, '2024-06-15', 200.00),
(2, 1, '2024-07-10', 150.00),
(3, 2, '2024-08-05', 300.00),
(4, 3, '2024-09-20', 180.00),
(5, 3, '2024-10-01', 220.00),
(6, 2, '2024-11-01', 90.00),
(7, 4, '2024-12-25', 110.00),
(8, 5, '2025-01-07', 130.00),
(9, 1, '2025-02-10', 250.00),
(10, 2, '2025-03-01', 180.00),
(11, 3, '2025-04-01', 200.00),
(12, 4, '2025-04-05', 150.00),
(13, 5, '2025-04-10', 300.00),
(14, 2, '2025-04-12', 100.00),
(15, 1, '2025-04-15', 220.00),
(16, 3, '2025-04-18', 250.00),
(17, 4, '2025-04-20', 190.00),
(18, 2, '2025-04-22', 130.00);
""")

cursor.execute("""
INSERT INTO itens_pedido (id, pedido_id, produto, quantidade, preco_unitario) VALUES
(1, 1, 'Mouse', 2, 50.00),
(2, 1, 'Teclado', 1, 100.00),
(3, 2, 'Monitor', 1, 150.00),
(4, 3, 'Mouse', 1, 50.00),
(5, 3, 'Cabo HDMI', 2, 40.00),
(6, 4, 'Webcam', 2, 90.00),
(7, 5, 'Teclado', 2, 110.00),
(8, 6, 'Cabo HDMI', 3, 30.00),
(9, 7, 'Fone de Ouvido', 1, 50.00),
(10, 8, 'Monitor', 1, 150.00),
(11, 9, 'Teclado', 1, 130.00),
(12, 10, 'Mouse', 2, 50.00),
(13, 11, 'Monitor', 1, 150.00),
(14, 12, 'Fone de Ouvido', 1, 50.00),
(15, 13, 'Teclado', 2, 110.00),
(16, 14, 'Mouse', 3, 50.00),
(17, 15, 'Webcam', 1, 90.00),
(18, 16, 'Monitor', 2, 150.00),
(19, 17, 'Teclado', 1, 110.00),
(20, 18, 'Cabo HDMI', 3, 30.00);
""")

# 3. Query: Total de pedidos por cliente
query_3 = """
SELECT
  c.nome,
  COUNT(p.id) AS total_pedidos
FROM clientes c
LEFT JOIN pedidos p ON c.id = p.cliente_id
GROUP BY c.id;
"""
df_total_pedidos_cliente = pd.read_sql_query(query_3, conn)

# Exibindo os resultados
display(HTML("<h3>Total de Pedidos por Cliente</h3>"))
display(df_total_pedidos_cliente)

# 4. Query: Ticket médio por cliente
query_4 = """
SELECT
  c.nome,
  ROUND(AVG(p.valor_total), 2) AS ticket_medio
FROM clientes c
JOIN pedidos p ON c.id = p.cliente_id
GROUP BY c.id;
"""
df_ticket_medio_cliente = pd.read_sql_query(query_4, conn)

# Exibindo os resultados
display(HTML("<h3>Ticket Médio por Cliente</h3>"))
display(df_ticket_medio_cliente)

# 5. Query: Top 5 produtos mais vendidos no último mês (abril de 2025)
query_5 = """
SELECT
  i.produto,
  SUM(i.quantidade) AS total_vendido,
  ROUND(SUM(i.quantidade * i.preco_unitario), 2) AS faturamento_total
FROM itens_pedido i
JOIN pedidos p ON i.pedido_id = p.id
WHERE p.data_pedido >= '2025-04-01' AND p.data_pedido < '2025-05-01'
GROUP BY i.produto
ORDER BY total_vendido DESC
LIMIT 5;
"""
df_top_produtos = pd.read_sql_query(query_5, conn)

# Exibindo os resultados
display(HTML("<h3>Top 5 Produtos Mais Vendidos em Quantidade no Último Mês (Abril de 2025)</h3>"))
display(df_top_produtos)

# 6. Query: Valor total de vendas por mês nos últimos 12 meses
query_6 = """
SELECT
  strftime('%Y-%m', p.data_pedido) AS mes,
  ROUND(SUM(p.valor_total), 2) AS total_vendas
FROM pedidos p
WHERE p.data_pedido >= DATE('now', '-12 months')
GROUP BY mes
ORDER BY mes DESC;
"""
df_total_vendas_mes = pd.read_sql_query(query_6, conn)

# Exibindo os resultados
display(HTML("<h3>Valor Total de Vendas por Mês (Últimos 12 Meses)</h3>"))
display(df_total_vendas_mes)

# Fechando a conexão com o banco de dados
conn.close()

,nome,total_pedidos
0,Lívia,4
1,Eduardo,5
2,Ana,4
3,Carlos,3
4,Mariana,2


,nome,ticket_medio
0,Lívia,205.0
1,Eduardo,160.0
2,Ana,212.5
3,Carlos,150.0
4,Mariana,215.0


,produto,total_vendido,faturamento_total
0,Teclado,3,330.0
1,Mouse,3,150.0
2,Monitor,3,450.0
3,Cabo HDMI,3,90.0
4,Webcam,1,90.0


,mes,total_vendas
0,2025-04,1540.0
1,2025-03,180.0
2,2025-02,250.0
3,2025-01,130.0
4,2024-12,110.0
5,2024-11,90.0
6,2024-10,220.0
7,2024-09,180.0
8,2024-08,300.0
9,2024-07,150.0
